In [19]:
import warnings
warnings.filterwarnings("ignore")

In [20]:
%run ../script/config.py

In [21]:
my_models_paths = !ls -d -1 ../data/models/*.txt

my_models_paths

['../data/models/4 - Baseline - Just join triples_e089e6d2328637218601e68ee1216f624e70d78f.txt',
 '../data/models/5 - Model - Template Based - roots_604049f5126c8e7723be2bcc42a5b3ba96fb29e8.txt',
 '../data/models/5 - Model - Template Based - roots_6dd871aa81bea6fcaf35b71faf932af2e82e17e6.txt',
 '../data/models/5 - Model - Template Based - roots_b6a7d881636c7be572e04d2d10a4523354324356.txt']

In [22]:
models_paths = my_models_paths + SUBMISSIONS_FILEPATHS
models_paths

['../data/models/4 - Baseline - Just join triples_e089e6d2328637218601e68ee1216f624e70d78f.txt',
 '../data/models/5 - Model - Template Based - roots_604049f5126c8e7723be2bcc42a5b3ba96fb29e8.txt',
 '../data/models/5 - Model - Template Based - roots_6dd871aa81bea6fcaf35b71faf932af2e82e17e6.txt',
 '../data/models/5 - Model - Template Based - roots_b6a7d881636c7be572e04d2d10a4523354324356.txt',
 '../data/webnlg2017/submissions/adaptCenter/ADAPTcentreWebNLGsubmission.txt',
 '../data/webnlg2017/submissions/melbourne/final_result.txt',
 '../data/webnlg2017/submissions/pkuwriter/PKUWriter_results.txt',
 '../data/webnlg2017/submissions/tilburg/nmt_test.out.ordered',
 '../data/webnlg2017/submissions/tilburg/smt_test.out.ordered',
 '../data/webnlg2017/submissions/tilburg/template_test.out.ordered',
 '../data/webnlg2017/submissions/uit-danglt-clnlp/Submission-UIT-DANGNT-CLNLP.txt',
 '../data/webnlg2017/submissions/upf/UPF_All_sent_final.txt',
 '../data/webnlg2017/submissions/baseline_sorted.txt']

In [23]:
model_results = {}

for model_path in models_paths:
    
    with open(model_path) as f:
        
        model_results[model_path] = [line[:-1] for line in f]

In [24]:
%run ../script/webnlg.py

In [25]:
import pandas as pd
pd.set_option('max_colwidth', 1000)
from nltk.translate.bleu_score import sentence_bleu

lex = WebNLGCorpus.load("test_with_lex")

s_multiwordexpressions = lex.mdf.m_subject.str.lower().str.replace('_', ' ').str.split().tolist()
o_multiwordexpressions = lex.mdf.m_object.str.lower().str.replace('_', ' ').str.split().tolist()

all_multiwordexpressions = s_multiwordexpressions + o_multiwordexpressions
all_multiwordexpressions = list(set([tuple(s) for s in all_multiwordexpressions]))

from nltk.tokenize.mwe import MWETokenizer

t = MWETokenizer(all_multiwordexpressions)

from textacy.preprocess import preprocess_text

def preprocess(ref):
    
    ref = preprocess_text(ref, no_accents=True, lowercase=True, no_punct=True)
    
    return t.tokenize(ref.split())


def print_entry_and_models(e):
    
    print(e)
    print('\n')
    
    idx = int(e.idx().split('_')[1])
    
    references = [preprocess(ref) for ref in e.ldf.ltext.tolist()]
    
    texts = []
    bleus = []
    
    for model_path in models_paths:
        
        model_text = model_results[model_path][idx]
        
        bleu = sentence_bleu(references, preprocess(model_text))
        
        texts.append(model_text)
        bleus.append(bleu)
        
    return pd.DataFrame({'models': models_paths,
                         'texts': texts,
                         'bleu': bleus}).sort_values('bleu', ascending=False)

In [26]:
test_with_lex = WebNLGCorpus.load('test_with_lex')

In [48]:
e = test_with_lex.sample()

df = print_entry_and_models(e)

Triple info: {'category': 'Politician', 'eid': 'Id1016', 'idx': '0_1015', 'ntriples': 1}

	Modified triples:

Gulf_War | commander | Colin_Powell


	Lexicalizations:

Colin Powell was a commander in the Gulf War.
Colin Powell was the commander in the Gulf War.




In [37]:
import spacy

nlp = spacy.load('en_core_web_lg')

In [38]:
doc = nlp(e.ldf.ltext.values[0])

In [39]:
from spacy import displacy

In [45]:
displacy.render(doc, jupyter=True, style='dep')

In [34]:
df

,bleu,models,texts
9,3.910803e-01,../data/webnlg2017/submissions/tilburg/template_test.out.ordered,aleksandr prudnikov was born on february 24th 1989 . aleksandr prudnikov plays for fc spartak moscow it is 185.0 ( centimetres ) high . it plays for fc rubin kazan .
7,3.064174e-01,../data/webnlg2017/submissions/tilburg/nmt_test.out.ordered,aleksandr prudnikov was born on february 24th 1989 in fc rubin kazan . fc spartak moscow was fc rubin kazan .
11,2.327512e-01,../data/webnlg2017/submissions/upf/UPF_All_sent_final.txt,"aleksandr prudnikov , born on february 24 , 1989 , plays for fc spartak moscow and fc rubin kazan . the height of aleksandr prudnikov is 185.0 centimetres ."
2,2.206773e-01,../data/models/5 - Model - Template Based - roots_6dd871aa81bea6fcaf35b71faf932af2e82e17e6.txt,Aleksandr Prudnikov plays for FC Spartak Moscow. Aleksandr Prudnikov was born on 1989-02-24. The height of Aleksandr Prudnikov is 185.0 (centimetres). Aleksandr Prudnikov plays for FC Rubin Kazan.
3,2.206773e-01,../data/models/5 - Model - Template Based - roots_b6a7d881636c7be572e04d2d10a4523354324356.txt,Aleksandr Prudnikov plays for FC Spartak Moscow. Aleksandr Prudnikov was born on 1989-02-24. The height of Aleksandr Prudnikov is 185.0 (centimetres). Aleksandr Prudnikov plays for FC Rubin Kazan.
5,1.969222e-01,../data/webnlg2017/submissions/melbourne/final_result.txt,aleksandr prudnikov was born in 1989-02-24 on 1989-02-24 and has 185.0 floors and is part of fc spartak moscow .
8,1.840993e-01,../data/webnlg2017/submissions/tilburg/smt_test.out.ordered,aleksandr prudnikov club fc spartak moscow aleksandr prudnikov was born on 1989-02-24 aleksandr prudnikov has a height of 185.0 ( centimetres ) aleksandr prudnikov club fc rubin kazan
1,5.749855e-78,../data/models/5 - Model - Template Based - roots_604049f5126c8e7723be2bcc42a5b3ba96fb29e8.txt,"Aleksandr Prudnikov plays for FC Spartak Moscow. , was born on 1989-02-24. The height of , is 185.0 (centimetres). , plays for FC Rubin Kazan."
0,2.659813e-78,../data/models/4 - Baseline - Just join triples_e089e6d2328637218601e68ee1216f624e70d78f.txt,Aleksandr Prudnikov club FC Spartak Moscow Aleksandr Prudnikov birthDate 1989-02-24 Aleksandr Prudnikov height 185.0 (centimetres) Aleksandr Prudnikov club FC Rubin Kazan
4,2.140702e-78,../data/webnlg2017/submissions/adaptCenter/ADAPTcentreWebNLGsubmission.txt,"the manager of alexiands , who was born on 1.2-24 , is fc sparta . he also plays for fc spartak mosca and fc runin kazan . he is also the member of alexiands , who was born on 1leksande , ."


In [29]:
%run ../script/data_alignment.py

import spacy

nlp = spacy.load('en_core_web_lg')

In [42]:
%run ../script/data_alignment.py

In [43]:
da = RootDataAlignmentModel(similarity_metric=similarity.token_sort_ratio)

In [49]:
first_lexicalization = e.ldf.ltext.values.tolist()[1]
# uses the first triple
first_triple = e.mdf[['m_subject', 'm_object']].to_dict(orient='records')[0]

da.render_aligned(nlp(first_lexicalization), first_triple)